In [48]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timezone


In [508]:
from datetime import datetime

epoch_time = datetime.now().timestamp()
print(int(epoch_time))
print(datetime.fromtimestamp(epoch_time, tz=timezone.utc7))


1695976424


AttributeError: type object 'datetime.timezone' has no attribute 'utc7'

In [499]:
from pprint import pprint
headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36 Edg/117.0.2045.41',
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'en-US,en;q=0.9,vi;q=0.8',
            'Connection': 'keep-alive',
            'Host': 'dchart-api.vndirect.com.vn'
        }
params = {
    'resolution': '1',
    'symbol': 'VNM',
    'from': str(int(epoch_time)-120 - (3600 * 24 )),
    'to': str(int(epoch_time) -120 ),
}
pprint(params)

{'from': '1695889815', 'resolution': '1', 'symbol': 'VNM', 'to': '1695976215'}


In [500]:
import requests
# response = requests.get(f'https://dchart-api.vndirect.com.vn/dchart/history?resolution=1&symbol=VNM&from={str(int(epoch_time)-120-86400)}&to={str(int(epoch_time)-120)}', headers=headers)
response = requests.get('https://dchart-api.vndirect.com.vn/dchart/history', headers=headers, params=params)
response.url

'https://dchart-api.vndirect.com.vn/dchart/history?resolution=1&symbol=VNM&from=1695889815&to=1695976215'

In [501]:
dic = response.json()

In [502]:
data = pd.DataFrame(dic)

# data['Date'] = data['t']
data['Date'] = data['t'].apply(lambda x: datetime.fromtimestamp(x, tz=timezone.utc))
data['o'] = data['o'].astype(float).apply(lambda x: x * 1000)
data['c'] = data['c'].astype(float).apply(lambda x: x * 1000)
data['h'] = data['h'].astype(float).apply(lambda x: x * 1000)
data['l'] = data['l'].astype(float).apply(lambda x: x * 1000)

In [503]:
# data = data.set_index('Date')
# data["Date"] = data.index.strftime("%Y/%m/%d %H:%M")
data = data.drop(['t', 's'], axis=1)
data.rename(columns = {
    'o':'open', 'c':'close',
    'h':'high', 'l': 'low', 
    'v': 'volume'
    }, inplace = True)
data['20wma'] = data['close'].rolling(window=20).mean()

In [504]:
data

,close,open,high,low,volume,Date,20wma
0,75100.0,75000.0,75200.0,75000.0,11900,2023-09-29 02:15:00+00:00,NaN
1,75100.0,75200.0,75200.0,75100.0,700,2023-09-29 02:16:00+00:00,NaN
2,75000.0,75200.0,75400.0,75000.0,17000,2023-09-29 02:17:00+00:00,NaN
3,75100.0,75100.0,75200.0,75000.0,2800,2023-09-29 02:18:00+00:00,NaN
4,75100.0,75000.0,75100.0,74900.0,18300,2023-09-29 02:19:00+00:00,NaN
...,...,...,...,...,...,...,...
186,75400.0,75500.0,75500.0,75400.0,5800,2023-09-29 07:26:00+00:00,75555.0
187,75300.0,75400.0,75400.0,75300.0,11500,2023-09-29 07:27:00+00:00,75525.0
188,75600.0,75300.0,75600.0,75300.0,8000,2023-09-29 07:28:00+00:00,75520.0
189,75600.0,75600.0,75600.0,75600.0,1000,2023-09-29 07:29:00+00:00,75515.0


# plot using  plotly

In [496]:
import plotly.graph_objects as go

In [497]:
fig = go.Figure(data=[go.Candlestick(x=data['Date'],
                open=data['open'],
                high=data['high'],
                low=data['low'],
                close=data['close'],)])

fig.add_trace(
    go.Scatter(
        x=data['Date'],
        y=data['20wma'],
        name='20 week ma',
        line=dict(color='grey', width=2, dash='dash')
    )
)
# Update the layout to allow zooming by day, month, and hour
fig.update_xaxes(
    rangeslider_visible=False,
    rangeselector=dict(
        # buttons=list([
        #     dict(count=1, label="1D", step="day", stepmode="todate"),
        #     dict(count=1, label="1M", step="month", stepmode="todate"),
        #     dict(step="all")
        # ]),
        bgcolor='lightgrey',  # Set the background color of the button area
        activecolor='blue'  # Set the color of the active button
    ),
    rangebreaks=[
        dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
        # dict(bounds=[16, 9.5], pattern="hour"),  # hide hours outside of 9.30am-4pm
    ]
)

fig.update_yaxes(type="log")

fig.update_layout(
    height=600,
    xaxis_rangeslider_visible=False, 
    template='plotly_dark',
    xaxis_title='Date', 
    yaxis_title='Price', 
    title='VNM', 
)

fig.show()

In [371]:
# Copyright (c) general_backbone. All rights reserved.
from vnquant.data import DataLoader
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import vnquant.utils.utils as utils
import pandas as pd
import numpy as np

def _vnquant_candle_stick_source(symbol,
                                 start_date, end_date,
                                 colors=['blue', 'red'],
                                 width=800, height=600,
                                 show_vol=True,
                                 data_source='VND',
                                 **kargs):
    loader = DataLoader(symbol, start_date, end_date, minimal=True, data_source=data_source)
    data = loader.download()
    symbol = list(data.columns.levels[1])[0]
    data.columns = ['high', 'low', 'open', 'close', 'adjust', 'volume']
    title = '{} stock price & volume from {} to {}'.format(symbol, start_date, end_date)
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02,
                        # subplot_titles=('Price', 'Volume'),
                        row_heights=[0.6, 0.4])

    fig.append_trace(go.Candlestick(
        x=data.index,
        open=data['open'], high=data['high'],
        low=data['low'], close=data['close'],
        increasing_line_color=colors[0],
        decreasing_line_color=colors[1]),
        row=1, col=1)

    if show_vol:
        fig.append_trace(go.Bar(
            x=data.index,
            y=data['volume'],
            name='Volume'),
            row=2, col=1)

    fig.update_layout(
        title=title,
        yaxis_title='Price',
        xaxis_title='Date',
        width=width,
        height=height,
        showlegend=False
    )

    fig.show()

def vnquant_candle_stick_source(
        symbol,
        start_date, end_date,
        colors=['blue', 'red'],
        width=800, height=600,
        data_source='VND',
        show_advanced = ['volume', 'macd', 'rsi'],
        **kargs
    ):
    '''
    This function is to visualize a candle stick stock index with advanced metrics
    Args:
        symbol (string): stock index
        start_date (string: 'yyyy-mm-dd'): start date
        end_date (string: 'yyyy-mm-dd'): end date
        colors (list: ['blue', 'red']): list colors of up and down candle
        width (int: 800): width of graph figure
        height (int: 600): height of graph figure
        data_source (string: 'VND'): data source to get stock price
        show_advanced (list: ['volume', 'macd', 'rsi']): list of advanced stock index to show up.
        
    Example:
        from vnquant import plot as pl
        pl.vnquant_candle_stick_source(
            symbol='TCB',
            title='TCB symbol from 2022-01-01 to 2022-10-01',
            xlab='Date', ylab='Price',
            start_date='2022-01-01',
            end_date='2022-10-01',
            data_source='CAFE',
            show_advanced = ['volume', 'macd', 'rsi']
        )
    '''
    
    loader = DataLoader(symbol, start_date, end_date, minimal=True, data_source=data_source)
    data = loader.download()
    symbol = list(data.columns.levels[1])[0]
    data.columns = ['high', 'low', 'open', 'close', 'adjust', 'volume']
    title = '{} stock price & volume from {} to {}'.format(symbol, start_date, end_date)
    num_indices = len(show_advanced)

    if num_indices == 3:
        r_price = 1
        r_volume = 2
        r_macd = 3
        r_rsi = 4
        w_macd = 1
        w_rsi = 1
        row_heights = [0.3, 0.3, 0.15, 0.15]

    if 'macd' not in show_advanced:
        r_price = 1
        r_volume = 2
        r_rsi = 3
        w_rsi = 1
        row_heights = [0.5, 0.3, 0.2]

    if 'rsi' not in show_advanced:
        r_price = 1
        r_volume = 2
        r_macd = 3
        w_macd = 1
        row_heights = [0.5, 0.3, 0.2]

    if ('rsi' not in show_advanced) and ('macd' not in show_advanced):
        r_price = 1
        r_volume = 2
        row_heights = [0.6, 0.4]

    fig = make_subplots(rows=num_indices + 1, cols=1, shared_xaxes=True, vertical_spacing=0.02,
                        # subplot_titles=('Price', 'Volume'),
                        row_heights=row_heights)

    fig.append_trace(
        go.Candlestick(
            x=data.index,
            open=data['open'], high=data['high'],
            low=data['low'], close=data['close'],
            increasing_line_color=colors[0],
            decreasing_line_color=colors[1]
        ),
        row=r_price, col=1
    )

    # Compute MACD:
    if 'macd' in show_advanced:
        # refers to: https://www.alpharithms.com/calculate-macd-python-272222/
        # Get the 26-day EMA of the closing price
        k = data['close'].ewm(span=12, adjust=False, min_periods=12).mean()
        # Get the 12-day EMA of the closing price
        d = data['close'].ewm(span=26, adjust=False, min_periods=26).mean()
        # Subtract the 26-day EMA from the 12-Day EMA to get the MACD
        macd = k - d
        # Get the 9-Day EMA of the MACD for the Trigger line
        macd_s = macd.ewm(span=9, adjust=False, min_periods=9).mean()
        # Calculate the difference between the MACD - Trigger for the Convergence/Divergence value
        macd_h = macd - macd_s
        # Add all of our new values for the MACD to the dataframe
        data['macd'] = data.index.map(macd)
        data['macd_h'] = data.index.map(macd_h)
        data['macd_s'] = data.index.map(macd_s)
        # Fast Signal (%k)
        fig.append_trace(
            go.Scatter(
                x=data.index,
                y=data['macd'],
                line=dict(color='#ff9900', width=w_macd),
                name='macd',
                showlegend=True,
                legendgroup='2',
            ), row=r_macd, col=1
        )
        # Slow signal (%d)
        fig.append_trace(
            go.Scatter(
                x=data.index,
                y=data['macd_s'],
                line=dict(color='#000000', width=w_macd),
                showlegend=True,
                legendgroup='2',
                name='signal'
            ), row=r_macd, col=1
        )
        # Colorize the histogram values
        colors = np.where(data['macd_h'] < 0, '#000', '#ff9900')
        # Plot the histogram
        fig.append_trace(
            go.Bar(
                x=data.index,
                y=data['macd_h'],
                name='histogram',
                marker_color=colors,
            ), row=r_macd, col=1
        )

    # Compute RSI:
    if 'rsi' in show_advanced:
        delta = data['close'].diff()
        up = delta.clip(lower=0)
        down = -1*delta.clip(upper=0)
        ema_up = up.ewm(com=13, adjust=False).mean()
        ema_down = down.ewm(com=13, adjust=False).mean()
        rs = ema_up/ema_down
        data['RSI'] = 100 - (100/(1+rs))

        fig.append_trace(
            go.Scatter(
                x=data.index, 
                y=data['RSI'], 
                name='RSI', 
                line=dict(width=w_rsi)
              ),
            row=r_rsi,
            col=1
        )

        fig.add_hline(y=70, line_dash="dot", row=r_rsi, col="all",
                  annotation_text="70%", 
                  annotation_position="bottom right")

        fig.add_hline(y=30, line_dash="dot", row=r_rsi, col="all",
                  annotation_text="30%", 
                  annotation_position="bottom right")

    # show volume    
    if 'volume' in show_advanced:
        fig.append_trace(
            go.Bar(
                x=data.index,
                y=data['volume'],
                name='Volume'
            ),
        row=r_volume, col=1)

    fig.update_layout(
        title=title,
        yaxis_title='Price',
        xaxis_title='Date',
        width=width,
        height=height,
        showlegend=True
    )

    fig.show()

def vnquant_candle_stick(data,
                          title=None,
                          xlab='Date', ylab='Price',
                          start_date=None, end_date=None,
                          colors=['blue', 'red'],
                          width=800, height=600,
                          data_source='VND',
                          show_advanced=['volume', 'macd', 'rsi'],
                          **kargs):
    '''
    This function is to visualize a candle stick stock index with advanced metrics
    Args:
        data (string or pandas DataFrame): stock data
        title (string: None): title of figure plot
        xlab (string: 'Date'): x label
        ylab (string: 'Price'): y label
        start_date (string: 'yyyy-mm-dd'): start date
        end_date (string: 'yyyy-mm-dd'): end date
        colors (list: ['blue', 'red']): list colors of up and down candle
        width (int: 800): width of graph figure
        height (int: 600): height of graph figure
        data_source (string: 'VND'): data source to get stock price belonging to ['VND', 'CAFE']
        show_advanced (list: ['volume', 'macd', 'rsi']): list of advanced stock index to show up. Each element belongs to ['volume', 'macd', 'rsi'] 
        
    Example:
        from vnquant import plot as pl
        pl.vnquant_candle_stick(
            data='TCB',
            title='TCB symbol from 2022-01-01 to 2022-10-01',
            xlab='Date', ylab='Price',
            start_date='2022-01-01',
            end_date='2022-10-01',
            data_source='CAFE',
            show_advanced = ['volume', 'macd', 'rsi']
        )
    '''
    # Download data from source
    if isinstance(data, str):
        vnquant_candle_stick_source(symbol=data, start_date=start_date, end_date=end_date,
                                     colors=colors, width=width,
                                     height=height, show_advanced=show_advanced,
                                     data_source=data_source)
    else:
        if 'volume' in show_advanced:
            assert utils._isOHLCV(data)
            defau_cols = ['high', 'low', 'open', 'close', 'volume']
            data = data[defau_cols].copy()
            data.columns = defau_cols
        else:
            assert utils._isOHLC(data)
            defau_cols = ['high', 'low', 'open', 'close']
            data = data[defau_cols].copy()
            data.columns = defau_cols

        x = data.index

        if not isinstance(x, pd.core.indexes.datetimes.DatetimeIndex):
            raise IndexError('index of dataframe must be DatetimeIndex!')

        if start_date is None:
            start_date = max(data.index)
        if end_date is None:
            end_date = max(data.index)

        fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02,
                            # subplot_titles=('Price', 'Volume'),
                            row_heights=[0.6, 0.4])

        fig.append_trace(go.Candlestick(
            x=x,
            open=data['open'], high=data['high'],
            low=data['low'], close=data['close'],
            increasing_line_color=colors[0],
            decreasing_line_color=colors[1]),
            row=1, col=1)

        if 'volume' in show_advanced:
            fig.append_trace(go.Bar(
                x=x,
                y=data['volume'],
                name='Volume'),
                row=2, col=1)

        fig.update_layout(
            title=title,
            yaxis_title=xlab,
            xaxis_title=ylab,
            showlegend=False
        )

        fig.show()

In [372]:
vnquant_candle_stick(data)